In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib
import re
import nltk
from tensorflow.keras.layers import BatchNormalization

# Unduh stopwords untuk Bahasa Indonesia
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

# Fungsi preprocessing
def preprocess_text(text):
    # Menghapus karakter yang tidak relevan dan menurunkan semua huruf menjadi kecil
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Menghapus stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords_indonesia])
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucifrr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
# Memuat data
data = pd.read_csv('../../models/ANN/data_set.csv')

# Membuat kolom 'Label' sebagai kombinasi dari 'Tipe_Emosi' dan 'Sumber_Emosi'
data['Label'] = data['Tipe_Emosi'] + "_" + data['Sumber_Emosi']

# Preprocessing teks
data['Input_Text'] = data['Level_Emosi'] + " " + data['Tipe_Emosi'] + " " + data['Sumber_Emosi']
data['Input_Text'] = data['Input_Text'].apply(preprocess_text)

# Membuat mapping label ke pertanyaan
label_to_questions = {}

for idx, row in data.iterrows():
    label = row['Label']
    questions = [q.strip() for q in row['Pertanyaan_Analisis'].split('|')]
    if label not in label_to_questions:
        label_to_questions[label] = questions
    else:
        # Jika label sudah ada, pastikan hanya menyimpan 5 pertanyaan unik
        existing_questions = set(label_to_questions[label])
        for q in questions:
            if q not in existing_questions and len(label_to_questions[label]) < 5:
                label_to_questions[label].append(q)
                existing_questions.add(q)

# Pastikan setiap label memiliki 5 pertanyaan unik
for label, questions in label_to_questions.items():
    label_to_questions[label] = list(dict.fromkeys(questions))[:5]

# Update TfidfVectorizer dan LabelEncoder
vectorizer = TfidfVectorizer(
    max_features=2000, 
    ngram_range=(1, 2), 
    stop_words=stopwords_indonesia,
    min_df=2  # Mengabaikan kata yang muncul kurang dari 2 kali
)
X = vectorizer.fit_transform(data['Input_Text']).toarray()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Label'])

# Periksa jumlah kelas dan total data
print(f"Jumlah total data: {len(data)}")
print(f"Jumlah kelas unik: {len(set(y))}")

# Pastikan test_size cukup untuk jumlah kelas
test_size = max(0.2, len(set(y)) / len(y))
print(f"Adjusted test size: {test_size}")

# Lakukan train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=42, stratify=y
)

# Periksa distribusi kelas pada train dan test set
print("Distribusi kelas pada train set:")
print(pd.Series(y_train).value_counts())

print("\nDistribusi kelas pada test set:")
print(pd.Series(y_test).value_counts())


Jumlah total data: 370
Jumlah kelas unik: 74
Adjusted test size: 0.2
Distribusi kelas pada train set:
33    4
20    4
44    4
21    4
31    4
     ..
16    4
28    4
43    4
6     4
56    4
Name: count, Length: 74, dtype: int64

Distribusi kelas pada test set:
53    1
26    1
57    1
33    1
45    1
     ..
61    1
16    1
29    1
22    1
12    1
Name: count, Length: 74, dtype: int64


d:\Project\SoulBabble\Phase 2\Machine Learning\venv\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [126]:
# Membuat model ANN yang disederhanakan
def build_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')  # Output layer
    ])
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=1e-3),
        metrics=['accuracy']
    )
    return model


In [127]:
# Callbacks untuk Early Stopping dan Model Checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras',  # Ubah ekstensi file menjadi .keras
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Membuat model
model = build_model(X_train.shape[1], len(set(y)))

# Melatih model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)


Epoch 1/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 7s 419ms/step - accuracy: 0.0000e+00 - loss: 4.3111
Epoch 1: val_loss improved from inf to 4.26402, saving model to best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0035 - loss: 4.3049 - val_accuracy: 0.0676 - val_loss: 4.2640
Epoch 2/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0000e+00 - loss: 4.2669
Epoch 2: val_loss improved from 4.26402 to 4.20796, saving model to best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0813 - loss: 4.2503 - val_accuracy: 0.1757 - val_loss: 4.2080
Epoch 3/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0625 - loss: 4.2099
Epoch 3: val_loss improved from 4.20796 to 4.12427, saving model to best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1874 - loss: 4.1899 - val_accuracy: 0.4324 - val_loss: 4.1243
Epoch 4/50
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4375 - loss: 4.0927
Epoch 4: val_loss improved from 4.12427 to 4.00122, sav

In [128]:
# Evaluasi model
score = model.evaluate(X_test, y_test, verbose=0)
print(f"Model Accuracy: {score[1]*100:.2f}%")

# Prediksi
y_pred = model.predict(X_test).argmax(axis=1)

# Pastikan target_names sesuai dengan kelas yang ada di y_test
unique_classes = sorted(set(y_test))  # Ambil kelas unik dari y_test
target_names = label_encoder.inverse_transform(unique_classes)  # Ambil nama kelas sesuai

# Laporan klasifikasi dengan labels yang sesuai
print(classification_report(
    y_test, 
    y_pred, 
    target_names=target_names,
    labels=unique_classes
))


Model Accuracy: 100.00%
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
                             precision    recall  f1-score   support

               Bahagia_Kafe       1.00      1.00      1.00         1
             Bahagia_Pantai       1.00      1.00      1.00         1
              Bahagia_Rumah       1.00      1.00      1.00         1
              Bahagia_Taman       1.00      1.00      1.00         1
              Bergairah_Gym       1.00      1.00      1.00         1
           Bergairah_Kantor       1.00      1.00      1.00         1
  Bersyukur_Kegiatan Sosial       1.00      1.00      1.00         1
            Bersyukur_Rumah       1.00      1.00      1.00         1
             Bingung_Kampus       1.00      1.00      1.00         1
             Bingung_Museum       1.00      1.00      1.00         1
       Bingung_Perpustakaan       1.00      1.00      1.00         1
            Bingung_Sekolah       1.00      1.00      1.00         1
               Cemas_Kantor       1.00  

In [132]:
# Fungsi untuk memprediksi dan menampilkan pertanyaan
def predict_and_display_questions(level_emosi, tipe_emosi, sumber_emosi):
    # Membuat label dari input triplet
    label = f"{tipe_emosi}_{sumber_emosi}"
    
    # Cek apakah label ada dalam mapping
    if label not in label_to_questions:
        print(f"Tidak ada pertanyaan yang tersedia untuk kombinasi: ({level_emosi}, {tipe_emosi}, {sumber_emosi})")
        return
    
    # Membuat teks input untuk prediksi
    input_text = f"{level_emosi} {tipe_emosi} {sumber_emosi}"
    processed_text = preprocess_text(input_text)
    
    # Vektorisasi
    vector = vectorizer.transform([processed_text]).toarray()
    
    # Prediksi
    prediction = model.predict(vector).argmax(axis=1)
    predicted_label = label_encoder.inverse_transform(prediction)[0]
    
    # Verifikasi apakah prediksi sesuai dengan input label
    if predicted_label != label:
        print(f"Peringatan: Prediksi label ({predicted_label}) tidak sesuai dengan input label ({label}).")
    
    # Menampilkan pertanyaan yang relevan
    questions = label_to_questions.get(predicted_label, [])
    if questions:
        print(f"\nPertanyaan untuk ({level_emosi}, {tipe_emosi}, {sumber_emosi}):")
        for idx, question in enumerate(questions, 1):
            print(f"{idx}. {question}")
    else:
        print("Tidak ada pertanyaan yang tersedia untuk label ini.")

# Contoh penggunaan
print("\nContoh Prediksi dan Penampilan Pertanyaan:")
# predict_and_display_questions("Sangat Buruk", "Kecewa", "Sekolah")
predict_and_display_questions("Sangat Baik", "Bersyukur", "Kegiatan Sosial")
# predict_and_display_questions("Sangat Baik", "Penuh Harapan", "Kampus") 



Contoh Prediksi dan Penampilan Pertanyaan:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

Pertanyaan untuk (Sangat Baik, Bersyukur, Kegiatan Sosial):
1. Apa yang membuat kamu merasa bersyukur di Kegiatan Sosial?
2. Apa yang bisa menyebabkan perasaan bersyukur kamu di Kegiatan Sosial?
3. Bagaimana situasi di Kegiatan Sosial mempengaruhi perasaan kamu yang bersyukur?
4. Apa langkah yang dapat kamu ambil untuk memperbaiki perasaan sangat buruk di Kegiatan Sosial?
5. Bagaimana cara kamu meningkatkan rasa bersyukur dalam kehidupan sehari-hari di Kegiatan Sosial?
